In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
## 模型的保存和导入
from keras.models import load_model
# 保存训练好的Tokenizer，和导入
import pickle
import jieba.analyse
import jieba
import re

Using TensorFlow backend.


In [2]:
# 导入已经训练好的模型
tnews = load_model('LSTM_tnews_better_model.h5')
thucnews = load_model('LSTM_thucnews_model.h5')
inews = load_model('inews_model.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [3]:
# tnews
# 读取训练集
train_file = open("./toutiao_category_train.txt", encoding='utf-8')
train_collection = train_file.readlines()
train_data = []
for i in train_collection:
    i = i.split("_!_")
    i[4] = i[4].strip()
    i[4] = " ".join(i[4].split(",")) + " " + " ".join(jieba.cut(re.sub(r'[^\w\s]','',i[3].strip()), cut_all=False))
    i.append(len(i[4].split(" ")))
    i.pop(0)
    i.pop(2)
    train_data.append(i)
train_df = pd.DataFrame(train_data)
train_df.columns = ["label_index", "label", "cutword", "cutwordnum"]
# 读取测试集
test_file = open("./toutiao_category_test.txt", encoding='utf-8')
test_collection = test_file.readlines()
test_data = []
for i in test_collection:
    i =i.split("_!_")
    i[4] = i[4].strip()
    i[4] = " ".join(i[4].split(",")) + " " + " ".join(jieba.cut(re.sub(r'[^\w\s]','',i[3].strip()), cut_all=False))
    i.append(len(i[4].split(" ")))
    i.pop(0)
    i.pop(2)
    test_data.append(i)
test_df = pd.DataFrame(test_data)
test_df.columns = ["label_index", "label", "cutword", "cutwordnum"]


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yinzh\AppData\Local\Temp\jieba.cache
Loading model cost 0.700 seconds.
Prefix dict has been built successfully.


In [4]:
## 对数据集的标签数据进行重新编码
train_y = train_df.label
test_y = test_df.label
le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1,1)
test_y = le.transform(test_y).reshape(-1,1)

## 对数据集的标签数据进行one-hot编码
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
test_y = ohe.transform(test_y).toarray()

max_words = 5000
max_len = 600
tok = Tokenizer(num_words=max_words)  ## 使用的最大词语数为5000
tok.fit_on_texts(train_df.cutword)

# 使用tok对验证数据集重新预处理
test_seq = tok.texts_to_sequences(test_df.cutword)
# 将每个序列调整为相同的长度
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)
# 对验证集进行预测
test_pre = tnews.predict(test_seq_mat)
print(metrics.classification_report(np.argmax(test_pre,axis=1),np.argmax(test_y,axis=1)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


              precision    recall  f1-score   support

           0       0.79      0.78      0.78      2972
           1       0.91      0.90      0.91      5292
           2       0.82      0.74      0.78      4734
           3       0.83      0.85      0.84      4101
           4       0.86      0.85      0.86      5886
           5       0.76      0.77      0.76      4071
           6       0.89      0.91      0.90      4183
           7       0.87      0.90      0.89      2579
           8       0.79      0.86      0.82      3516
           9       0.92      0.93      0.92      5544
          10       0.75      0.76      0.76       937
          11       0.85      0.85      0.85      6292
          12       0.76      0.75      0.75      3325
          13       0.79      0.78      0.79      3972
          14       0.00      0.00      0.00         0

    accuracy                           0.84     57404
   macro avg       0.77      0.78      0.77     57404
weighted avg       0.84   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [11]:
tnews_file = open('tnews_result.txt', 'w')
for i in np.argmax(test_pre,axis=1):
    t = -1
    if i == 0:
        t = 115
    elif i == 1:
        t = 107
    elif i == 2:
        t = 101
    elif i == 3:
        t = 108
    elif i == 4:
        t = 102
    elif i == 5:
        t = 104
    elif i == 6:
        t = 116
    elif i == 7:
        t = 106
    elif i == 8:
        t = 110
    elif i == 9:
        t = 103
    elif i == 10:
        t = 100
    elif i == 11:
        t = 109
    elif i == 12:
        t = 112
    elif i == 13:
        t = 113
    else:
        print("Error")
        break
    tnews_file.write("{}\n".format(t))
tnews_file.close()

In [12]:
# thucnews
# 读取训练集
train_file = open("./thucnews_train.txt", encoding='utf-8')
train_collection = train_file.readlines()
train_data = []
for i in train_collection:
    i = i.split("_!_")
    if len(i) < 4:
        continue
    i[3] = " ".join(jieba.analyse.textrank(i[3], topK=30))
    i.append(len(i[3].split()))
    i.pop(2) # 去除ID
    train_data.append(i)
train_df = pd.DataFrame(train_data)
train_df.columns = ["label_index", "label", "cutword", "cutwordnum"]

# 读取测试集
test_file = open("./thucnews_test.txt", encoding='utf-8')
test_collection = test_file.readlines()
test_data = []
for i in test_collection:
    i = i.split("_!_")
    if len(i) < 4:
        continue
    i[3] = " ".join(jieba.analyse.textrank(i[3], topK=30))
    i.append(len(i[3].split()))
    i.pop(2) # 去除ID
    test_data.append(i)
test_df = pd.DataFrame(test_data)
test_df.columns = ["label_index", "label", "cutword", "cutwordnum"]

In [13]:
## 对数据集的标签数据进行重新编码
train_y = train_df.label
test_y = test_df.label
le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1,1)
test_y = le.transform(test_y).reshape(-1,1)

## 对数据集的标签数据进行one-hot编码
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
test_y = ohe.transform(test_y).toarray()

max_words = 5000
max_len = 600
tok = Tokenizer(num_words=max_words)  ## 使用的最大词语数为5000
tok.fit_on_texts(train_df.cutword)

# 使用tok对验证数据集重新预处理
test_seq = tok.texts_to_sequences(test_df.cutword)
# 将每个序列调整为相同的长度
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)
# 对验证集进行预测
test_pre = thucnews.predict(test_seq_mat)
print(metrics.classification_report(np.argmax(test_pre,axis=1),np.argmax(test_y,axis=1)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


              precision    recall  f1-score   support

           0       0.95      0.97      0.96       671
           1       0.92      0.77      0.84       530
           2       0.82      0.71      0.76       168
           3       0.72      0.82      0.77        38
           4       0.84      0.79      0.81       102
           5       0.86      0.87      0.87       230
           6       0.64      0.60      0.62        60
           7       0.73      0.87      0.80       247
           8       0.15      0.50      0.24         4
           9       0.72      0.83      0.77        98
          10       0.72      0.80      0.76       224
          11       0.86      0.89      0.88       801
          12       0.91      0.86      0.89       827
          13       0.75      0.78      0.76       180

    accuracy                           0.86      4180
   macro avg       0.76      0.79      0.77      4180
weighted avg       0.86      0.86      0.86      4180



In [14]:
# count = 0
# for i in np.argmax(test_y,axis=1):
#     print(i)
#     count += 1
#     if count == 100:
#         break

13
0
0
1
7
0
13
4
5
0
0
7
7
0
10
10
12
11
1
10
1
13
11
11
0
5
0
13
0
12
1
6
11
11
7
12
2
12
12
10
0
1
11
12
1
12
11
11
0
0
12
11
12
0
4
0
11
4
11
11
0
0
11
12
7
2
12
2
13
11
0
11
0
11
11
0
0
1
13
1
7
0
0
12
2
0
1
5
0
11
11
11
1
1
0
7
0
11
2
0


In [15]:
thucnews_file = open('thucnews_result.txt', 'w')
for i in np.argmax(test_pre,axis=1):
    thucnews_file.write("{}\n".format(i))
thucnews_file.close()

In [16]:
# 读取训练集
train_file = open("./inews_train.txt", encoding='utf-8')
train_collection = train_file.readlines()
train_data = []
for i in train_collection:
    i = i.split("_!_")
    # 提取30个关键词 + " " + 标题分词
    i[2] = " ".join(jieba.analyse.textrank(i[3], topK=30)) + " " + " ".join(jieba.cut(re.sub(r'[^\w\s]','',i[2].strip()), cut_all=False))
    i.append(len(i[2].split()))
    i.pop(1) # 去除ID
    i.pop(2)
    train_data.append(i)
train_df = pd.DataFrame(train_data)
train_df.columns = ["label", "cutword", "cutwordnum"]

# 读取测试集
test_file = open("./inews_test.txt", encoding='utf-8')
test_collection = test_file.readlines()
test_data = []
for i in test_collection:
    i = i.split("_!_")
    i[2] = " ".join(jieba.analyse.textrank(i[3], topK=30)) + " " + " ".join(jieba.cut(re.sub(r'[^\w\s]','',i[2].strip()), cut_all=False))
    i.append(len(i[2].split()))
    i.pop(1) # 去除ID
    i.pop(2)
    test_data.append(i)
test_df = pd.DataFrame(test_data)
test_df.columns = ["label", "cutword", "cutwordnum"]

In [17]:
## 对数据集的标签数据进行重新编码
train_y = train_df.label
test_y = test_df.label
le = LabelEncoder()
train_y = le.fit_transform(train_y).reshape(-1,1)
test_y = le.transform(test_y).reshape(-1,1)

## 对数据集的标签数据进行one-hot编码
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
test_y = ohe.transform(test_y).toarray()

max_words = 5000
max_len = 600
tok = Tokenizer(num_words=max_words)  ## 使用的最大词语数为5000
tok.fit_on_texts(train_df.cutword)

# 使用tok对验证数据集重新预处理
test_seq = tok.texts_to_sequences(test_df.cutword)
# 将每个序列调整为相同的长度
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)
# 对验证集进行预测
test_pre = inews.predict(test_seq_mat)
print(metrics.classification_report(np.argmax(test_pre,axis=1),np.argmax(test_y,axis=1)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


              precision    recall  f1-score   support

           0       0.63      0.47      0.54       131
           1       0.72      0.80      0.75       444
           2       0.82      0.78      0.80       425

    accuracy                           0.75      1000
   macro avg       0.72      0.68      0.70      1000
weighted avg       0.75      0.75      0.74      1000



In [18]:
# count = 0
# for i in np.argmax(test_y,axis=1):
#     print(i)
#     count += 1
#     if count == 100:
#         break

1
1
2
1
1
0
2
2
0
2
2
2
1
1
1
1
2
1
2
1
1
1
1
2
2
2
2
2
0
2
1
2
1
1
2
1
0
0
1
2
1
1
1
1
2
2
2
2
1
1
0
1
0
2
1
1
2
2
1
2
1
2
1
1
1
2
1
1
2
2
1
1
2
1
2
1
2
2
2
2
0
0
2
1
0
1
1
2
1
2
2
2
0
1
1
1
1
1
2
0


In [19]:
inews_file = open('inews_result.txt', 'w')
for i in np.argmax(test_pre,axis=1):
    inews_file.write("{}\n".format(i))
inews_file.close()